<a href="https://www.kaggle.com/code/ebruiserisobay/association-rule-based-recommender-system-retail?scriptVersionId=183564205" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Association Rule Based Recommender System (Retail)

## 1. Business Problem

Below are the basket details of 3 different users. The most suitable product recommendations for these basket details have been made using association rules.

The decision rules are derived from 2010-2011 Germany customers.

* The product ID in User 1's basket: 21987
* The product ID in User 2's basket: 22747

## 2. About Dataset

The Online Retail II dataset contains the online sales transactions of a UK-based retail company between 01/12/2009 and 09/12/2011. The company's product catalog includes gift items, and it is known that most of its customers are wholesalers.

* **InvoiceNo:** Invoice Number (If this code starts with C, it indicates the transaction was canceled)
* **StockCode:** Product code (Unique for each product)
* **Description:** Product name
* **Quantity:** Quantity of products (The number of each product sold in the invoices)
* **InvoiceDate:** Invoice date
* **UnitPrice:** Invoice price (in Sterling)
* **CustomerID:** Unique customer number
* **Country:** Country name


## 3. Data Preparing & Understanding

In [1]:
import pandas as pd
!pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

In [2]:
df_ = pd.read_excel('/kaggle/input/retail-dataset/online_retail_II.xlsx') #upload the dataset

In [3]:
df = df_.copy() # copy the dataset to protect original dataset

In [4]:
df.head() #display first 5 rows

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
df.shape #display the shape of the dataset

(525461, 8)

In [6]:
df.info() #info about dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [7]:
# POST is the price added to each invoice, it does not refer to the product.

# Observation units with StockCode POST have been dropped :

df = df[df["StockCode"] != "POST"]

In [8]:
# Number of observation units with null values:

df.isnull().sum()

Invoice             0
StockCode           0
Description      2925
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107884
Country             0
dtype: int64

In [9]:
# Drop observation units containing null values:

df.dropna(inplace=True)

In [10]:
df.isnull().sum() #check again

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [11]:
# "C" means canceling the invoice.

# Removing the values with C in Invoice from the data set:

df = df[~ df["Invoice"].str.contains("C", na = False)]

In [12]:
# Filter observation units with a Price and Quantity value greater than zero:

df = df[df["Price"] > 0]

df = df[df["Quantity"] > 0]

In [13]:
df.describe() #descriptive statistics

,Quantity,InvoiceDate,Price,Customer ID
count,406926.000000,406926,406926.000000,406926.000000
mean,13.604788,2010-07-01 09:58:01.902655744,3.251350,15373.459877
min,1.000000,2009-12-01 07:45:00,0.001000,12346.000000
25%,2.000000,2010-03-26 14:01:00,1.250000,14006.000000
50%,5.000000,2010-07-09 15:31:00,1.950000,15325.000000
75%,12.000000,2010-10-14 17:09:00,3.750000,16814.000000
max,19152.000000,2010-12-09 20:01:00,10953.500000,18287.000000
std,96.927416,NaN,34.720873,1677.269478


In [14]:
# Defining the functions necessary for suppressing outliers

# find outlier thresholds:

def outlier_thresholds(dataframe,variable):
    quartile_1 = dataframe[variable].quantile(0.01)
    quartile_3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile_3 - quartile_1
    up_limit = quartile_3 + 1.5 * interquantile_range
    low_limit = quartile_1 - 1.5 * interquantile_range
    return low_limit, up_limit


In [15]:
# apply outlier thresholds:

def replace_with_threshold(dataframe,variable):
    low_limit,up_limit = outlier_thresholds(dataframe,variable)
    dataframe.loc[(dataframe[variable] < low_limit),variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit),variable] = up_limit

In [16]:
# Let's suppress the outliers of the variables which need:

for col in ["Quantity","Price"]:
    replace_with_threshold(df, col)

/tmp/ipykernel_18/2199502910.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-213.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit),variable] = low_limit


In [17]:
df.describe() # check descriptive statistics again

,Quantity,InvoiceDate,Price,Customer ID
count,406926.000000,406926,406926.000000,406926.000000
mean,11.922683,2010-07-01 09:58:01.902655744,2.964390,15373.459877
min,1.000000,2009-12-01 07:45:00,0.001000,12346.000000
25%,2.000000,2010-03-26 14:01:00,1.250000,14006.000000
50%,5.000000,2010-07-09 15:31:00,1.950000,15325.000000
75%,12.000000,2010-10-14 17:09:00,3.750000,16814.000000
max,358.500000,2010-12-09 20:01:00,31.440000,18287.000000
std,28.748450,NaN,3.057273,1677.269478


In [18]:
df.head() #display first 5 rows again

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [19]:
# filtering German customers and creating a new dataframe:

df_de = df[df["Country"] == "Germany"]

df_de.head() #display first 5 rows 

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
579,489526,85049E,SCANDINAVIAN REDS RIBBONS,12.0,2009-12-01 11:50:00,1.25,12533.0,Germany
580,489526,21976,PACK OF 60 MUSHROOM CAKE CASES,24.0,2009-12-01 11:50:00,0.55,12533.0,Germany
581,489526,21498,RED SPOTS WRAP,25.0,2009-12-01 11:50:00,0.42,12533.0,Germany
582,489526,22077,6 RIBBONS RUSTIC CHARM,12.0,2009-12-01 11:50:00,1.65,12533.0,Germany
583,489526,84946,ANTIQUE SILVER TEA GLASS ETCHED,12.0,2009-12-01 11:50:00,1.25,12533.0,Germany


In [20]:
# how many of which products were purchased:

df_de.groupby(["Invoice","Description"]).agg({"Quantity":"sum"}).head()

Quantity
Invoice Description                               
489526  6 RIBBONS RUSTIC CHARM                12.0
        ANTIQUE SILVER TEA GLASS ETCHED       12.0
        BIRD DECORATION RED SPOT              24.0
        BLUE/BROWN DOTS RUFFLED UMBRELLA       3.0
        EDWARDIAN PARASOL BLACK                6.0

# 4. Preparing "Invoice - Product Matrix"

In [21]:
# Defining "a invoice - product pivot table" function:

def create_invoice_product_df(dataframe, id = False):
    if id:
        return dataframe.pivot_table(index = "Invoice", columns = "StockCode", values = "Quantity" ,aggfunc={'Quantity': 'sum'}).\
        fillna(0).\
        map(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.pivot_table(index = "Invoice", columns = "Description", values = "Quantity",aggfunc={'Quantity': 'sum'} ).\
        fillna(0).\
        map(lambda x: 1 if x > 0 else 0)

In [22]:
# apply the function with "Description":

de_inv_pro_df = create_invoice_product_df(df_de)

de_inv_pro_df.head() # display first 5 columns 

Description,3 STRIPEY MICE FELTCRAFT,CHERRY BLOSSOM DECORATIVE FLASK,FLAMINGO LIGHTS,RED/WHITE DOT MINI CASES,SET 2 TEA TOWELS I LOVE LONDON,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 MINI TOADSTOOL PEGS,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE RED SPOTTY,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE RED SPOTTY,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,3 PIECE SPACEBOY COOKIE CUTTER SET,3 STRIPEY MICE FELTCRAFT,36 DOILIES DOLLY GIRL,36 DOILIES SPACEBOY DESIGN,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE RED SPOTTY,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D HEARTS HONEYCOMB PAPER GARLAND,3D STICKERS CHRISTMAS STAMPS,3D STICKERS TRADITIONAL CHRISTMAS,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 GOLD FLOCK CHRISTMAS BALLS,4 LAVENDER BOTANICAL DINNER CANDLES,4 TRADITIONAL SPINNING TOPS,4 VANILLA BOTANICAL CANDLES,4 WILDFLOWER BOTANICAL CANDLES,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,6 CHOCOLATE LOVE HEART T-LIGHTS,6 CROCHET STRAWBERRIES,6 EGG HOUSE PAINTED WOOD,6 RIBBONS ELEGANT CHRISTMAS,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,6 RIBBONS SHIMMERING PINKS,6 ROCKET BALLOONS,6 SEGMENT COLOURED SNACK BOWL,60 CAKE CASES DOLLY GIRL DESIGN,60 CAKE CASES VINTAGE CHRISTMAS,60 TEATIME FAIRY CAKE CASES,72 CAKE CASES DOLLY GIRL DESIGN,72 CAKE CASES VINTAGE CHRISTMAS,72 SWEETHEART FAIRY CAKE CASES,75 GREEN FAIRY CAKE CASES,ABSTRACT CIRCLE JOURNAL,ABSTRACT CIRCLES POCKET BOOK,ABSTRACT CIRCLES SKETCHBOOK,ADVENT CALENDAR GINGHAM SACK,AFGHAN SLIPPER SOCK PAIR,AIRLINE BAG VINTAGE JET SET BROWN,AIRLINE BAG VINTAGE JET SET RED,AIRLINE BAG VINTAGE JET SET WHITE,AIRLINE BAG VINTAGE TOKYO 78,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ALPHABET HEARTS STICKER SHEET,ALPHABET STENCIL CRAFT,ANGEL DECORATION PAINTED ZINC,ANTIQUE CREAM CUTLERY CUPBOARD,ANTIQUE CREAM CUTLERY SHELF,ANTIQUE GLASS DRESSING TABLE POT,ANTIQUE GLASS PEDESTAL BOWL,ANTIQUE SILVER TEA GLASS ENGRAVED,ANTIQUE SILVER TEA GLASS ETCHED,APPLE BATH SPONGE,APRON APPLE DELIGHT,ASSORTED CAKES FRIDGE MAGNETS,ASSORTED CHEESE FRIDGE MAGNETS,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR LIZARD SUCTION HOOK,ASSORTED COLOUR MINI CASES,ASSORTED COLOUR SET 6 TEASPOONS,ASSORTED COLOUR SILK COIN PURSE,ASSORTED COLOURS SILK FAN,ASSORTED CREEPY CRAWLIES,ASSORTED EASTER GIFT TAGS,"ASSORTED FLOWER COLOUR ""LEIS""",ASSORTED ICE CREAM FRIDGE MAGNETS,ASSORTED MONKEY SUCTION CUP HOOK,ASSORTED SANSKRIT MINI NOTEBOOK,ASSORTED TUTTI FRUTTI FOB NOTEBOOK,ASSORTED TUTTI FRUTTI ROUND BOX,ASSORTED TUTTI FRUTTI SMALL PURSE,ASSTD DESIGN 3D PAPER STICKERS,ASSTD DESIGN BUBBLE GUM RING,ASSTD DESIGN POP ART SLIM PEN,ASSTD DESIGN RACING CAR PEN,ASSTD FRUIT+FLOWERS FRIDGE MAGNETS,Adjustment by john on 26/01/2010 16,Adjustment by john on 26/01/2010 17,BABUSHKA LIGHTS STRING OF 10,BABY BOOM RIBBONS,BAG 125g SWIRLY MARBLES,BAG 250g SWIRLY MARBLES,BAG 500g SWIRLY MARBLES,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BALLOON ART MAKE YOUR OWN FLOWERS,BALLOON WATER BOMB PACK OF 35,BALLOONS WRITING SET,BASKET OF TOADSTOOLS,BATH BUILDING BLOCK WORD,BATH DUCK WATERING CAN,BATHROOM METAL SIGN,BATHROOM SET LOVE HEART DESIGN,BEACH HUT KEY CABINET,BEADED LOVE HEART JEWELLERY SET,BEWARE OF THE CAT DOORMAT,BEWARE OF THE CAT METAL SIGN,BIG DOUGHNUT FRIDGE MAGNETS,BIRD DECORATION GREEN SPOT,BIRD DECORATION RED RETROSPOT,BIRD DECORATION RED SPOT,BIRD HOUSE HOT WATER BOTTLE,BIRDCAGE DECORATIO

In [23]:
# Creating a function to check product names:

def check_id(dataframe,stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].\
    values[0].\
    tolist()
    print(product_name)

In [24]:
# apply the function with "StockCode":

de_inv_pro_df = create_invoice_product_df(df_de,id = True)

de_inv_pro_df.head() # display first 5 columns 

StockCode,10002,10125,10135,11001,15034,15036,15039,16012,16016,16033,16045,16052,16236,16237,16238,16259,17003,20617,20619,20622,20652,20653,20654,20658,20661,20665,20668,20671,20672,20674,20675,20676,20677,20679,20681,20682,20684,20685,20689,20692,20696,20702,20703,20711,20712,20713,20716,20717,20718,20719,20723,20724,20725,20726,20727,20728,20729,20730,20734,20738,20749,20750,20751,20752,20754,20755,20756,20758,20759,20760,20761,20762,20764,20766,20767,20768,20769,20770,20771,20772,20794,20798,20799,20816,20828,20829,20857,20864,20866,20867,20868,20871,20886,20914,20957,20961,20963,20964,20966,20967,20971,20972,20973,20974,20975,20977,20978,20979,20981,20982,20983,20984,20985,20990,20997,20998,21000,21001,21002,21011,21025,21026,21027,21033,21035,21039,21041,21042,21051,21056,21058,21059,21064,21067,21069,21070,21077,21078,21080,21082,21084,21085,21086,21088,21090,21091,21092,21094,21096,21098,21099,21100,21106,21108,21110,21114,21115,21116,21117,21121,21122,21124,21125,21126,21127,21130,21135,21136,21137,21144,21153,21154,21155,21156,21157,21162,21163,21164,21165,21166,21179,21186,21188,21190,21191,21197,21202,21203,21204,21205,21206,21207,21210,21211,21212,21213,21215,21216,21217,21218,21219,21220,21221,21222,21223,21224,21226,21231,21232,21238,21239,21240,21241,21242,21243,21244,21245,21246,21247,21248,21249,21251,21258,21259,21260,21263,21269,21270,21272,21275,21276,21278,21281,21282,21284,21285,21286,21291,21307,21319,21320,21324,21327,21328,21329,21330,21348,21354,21355,21357,...,84948,84949,84969,84978,84987,84988,84991,84992,85006,85007,85015,85016,85038,85054,85055,85059,85062,85064,85067,85068,85087,85090,85093,85119,85174,85175,85178,85200,85215,85216,85217,85221,85227,90104,90167,15044A,15044B,15044D,15056BL,15056N,15056P,15058A,15060B,16156S,16161G,16161P,16161U,16168M,16169C,16169F,16169K,16169M,16169N,16169P,16256B,17091A,17109D,35095A,35471A,35471D,35597B,35598B,35598D,35599B,35599D,35600D,35637A,35809A,35810A,35810B,35811A,35833G,35910A,37482P,40018F,40046A,46000M,46000S,46126A,47367B,47504K,47518F,47556B,47559B,47566B,47567B,47570B,47585A,47586A,47587A,47590A,47590B,47591A,47591B,47591D,47592A,47593A,47594A,47594B,47599A,47599B,48173C,51014L,62094B,72140F,72351A,72351B,72750B,72760B,79030A,79030D,79030G,79190B,79190D,79191B,79191C,81950B,81952B,81952V,82494L,82613A,82613B,82613D,82616C,84029E,84029G,84030E,84031A,84078A,84086C,84247C,84247E,84247G,84247K,84247L,84247N,84279B,84279P,84341B,84406B,84429A,84437A,84507C,84508B,84509A,84509B,84509G,84510A,84510B,84520B,84529C,84531B,84533B,84534B,84536A,84569B,84569C,84575B,84597C,84625C,84661A,84661B,84661C,84709B,84797A,84797B,84849B,84849D,84859A,84899E,84899F,84909A,84912A,84912B,84913A,84927A,84932A,84932B,84932C,84932D,84968A,84968B,84968C,84968D,84968E,84968F,84970L,84970S,84989A,84997A,84997B,84997C,84997D,85014B,85014C,85017A,85017B,85017C,85031A,85031B,85032A,85032B,85032D,85040B,85049A,85049B,85049C,85049D,85049E,85049F,85049G,85049H,85098B,85099B,85099C,85099F,85121C,85123A,85129A,85132A,85132B,85132C,85136A,85159B,85162A,85162B,85162C,85164B,85169C,85169D,85184D,85199L,85199S,85205B,85206A,85206B,85226A,85226C,85231B,85231E,85231L,85232A,85232B,85232D,90018A,90019A,90200E,ADJUST,M,PADS
Invoice,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
489526,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
# checking a product name with random StockCode:

check_id(df_de,10002)

['INFLATABLE POLITICAL GLOBE ']


## 5. Association Rules Analysis

In [26]:
# Apriori Algorithm:

frequent_itemsets = apriori(de_inv_pro_df,min_support = 0.01,use_colnames = True) # applying

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [27]:
frequent_itemsets.sort_values("support",ascending=False).head() #sorting

,support,itemsets
265,0.302115,(22326)
22,0.226586,(20719)
267,0.202417,(22328)
151,0.187311,(21731)
77,0.163142,(21238)


In [28]:
# creating association rules:

rules = association_rules(frequent_itemsets,metric = "support",min_threshold = 0.01)

In [29]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(15036),(20719),0.018127,0.226586,0.015106,0.833333,3.677778,0.010998,4.640483,0.741538
1,(20719),(15036),0.226586,0.018127,0.015106,0.066667,3.677778,0.010998,1.052007,0.941406
2,(15036),(20724),0.018127,0.105740,0.012085,0.666667,6.304762,0.010168,2.682779,0.856923
3,(20724),(15036),0.105740,0.018127,0.012085,0.114286,6.304762,0.010168,1.108566,0.940878
4,(22328),(15036),0.202417,0.018127,0.012085,0.059701,3.293532,0.008415,1.044214,0.873106


## 6. Association Rules Learning (ARL) Recommender 

In [30]:
# defining arl_recommender function:

def arl_recommender(rules_df,product_id,rec_count=1):
    sorted_rules = rules_df.sort_values("lift",ascending = False)
    recommendation_list = []
    for i,product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
    return recommendation_list[0:rec_count]


In [31]:
# product id: 21987

# Recommending a product for the "user 1":

arl_recommender(rules,21987,1)

[21988]

In [32]:
# checking the product name for "user 1":

check_id(df_de,21988)

['PACK OF 6 SKULL PAPER PLATES']


In [33]:
# product id: 22747

# Recommending a product for the "user 2":

arl_recommender(rules,22747,1)

[22745]

In [34]:
# checking the product name for "user 2":

check_id(df_de,22745)

["POPPY'S PLAYHOUSE BEDROOM "]
